# Start

In [1]:
import importlib
import os
# import pandas as pd
# import json
# from datetime import datetime
from pathlib import Path
# from io import StringIO

from langchain_core.tools import Tool, StructuredTool
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, SystemMessage

from bsm_multi_agents.graph.state import WorkflowState

In [2]:
# from bsm_multi_agents.agents import utils
# importlib.reload(utils)
# from bsm_multi_agents.agents.utils import get_tool_result_from_messages,print_resp

In [3]:
cwd = Path.cwd()
project_path = cwd.parent.parent
project_path

PosixPath('/Users/yifanli/Github/model_doc_automation')

# Graph

In [161]:
from langgraph.graph import StateGraph, END, START

from bsm_multi_agents.graph.state import WorkflowState
from bsm_multi_agents.agents.pricing_calculator_nodes import (
    pricing_calculator_agent_node,
    pricing_calculator_tool_node,
)

graph = StateGraph(WorkflowState)

graph.add_node("pricing_calculator_tool", pricing_calculator_tool_node)
graph.add_node("pricing_validator_agent", pricing_validator_agent_node)

graph.add_edge(START, "pricing_calculator_tool")
graph.add_edge("pricing_calculator_tool", "pricing_validator_agent")
graph.add_edge("pricing_validator_agent", END)

app = graph.compile()

In [162]:
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
init_state = WorkflowState(
    csv_file_path=csv_file_path,
    output_dir=output_dir,
    server_path=server_path,
    # "remaining_steps": 10,
    # "messages": [HumanMessage(content=f"Load CSV from: {file_path}")],
)

final_state = app.invoke(
    init_state,
    config={"configurable": {"thread_id": "run-1"}}
)

In [163]:
final_state

{'messages': [],
 'csv_file_path': '/Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv',
 'output_dir': '/Users/yifanli/Github/model_doc_automation/data/cache',
 'server_path': '/Users/yifanli/Github/model_doc_automation/src/bsm_multi_agents/mcp/server.py',
 'greeks_results_path': '/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_greeks_results.csv',
 'validate_results_path': '/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_greeks_results_validate_results.csv',
 'errors': [],
 'remaining_steps': 0}

# Calculator

## MCP Tool Level

### calculate_bsm_to_file

In [45]:
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [46]:
from bsm_multi_agents.mcp import pricing_calculator
importlib.reload(pricing_calculator)
from bsm_multi_agents.mcp.pricing_calculator import calculate_bsm_to_file

In [47]:
calculate_bsm_to_file(csv_file_path, output_dir)

/Users/yifanli/Github/model_doc_automation/src/bsm_multi_agents/mcp/pricing_calculator.py:11: RuntimeWarning: invalid value encountered in sqrt
  d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
/Users/yifanli/Github/model_doc_automation/src/bsm_multi_agents/mcp/pricing_calculator.py:12: RuntimeWarning: invalid value encountered in sqrt
  d2 = d1 - sigma * np.sqrt(T)
/Users/yifanli/Github/model_doc_automation/src/bsm_multi_agents/mcp/pricing_calculator.py:11: RuntimeWarning: invalid value encountered in log
  d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))


'/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_bsm_results.csv'

### calculate_greeks_to_file

In [48]:
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [49]:
from bsm_multi_agents.mcp import pricing_calculator
importlib.reload(pricing_calculator)
from bsm_multi_agents.mcp.pricing_calculator import calculate_greeks_to_file

In [50]:
calculate_greeks_to_file(csv_file_path, output_dir)

'/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_greeks_results.csv'

## MCP Server Level

## Call MCP Level

### call_mcp_tool_async

In [51]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [52]:
from bsm_multi_agents.agents import mcp_client
importlib.reload(mcp_client)
from bsm_multi_agents.agents.mcp_client import call_mcp_tool_async

In [53]:
res = await call_mcp_tool_async(
    tool_name="calculate_bsm_to_file",
    server_script_path=server_path,
    arguments={
        "input_path": csv_file_path,
        "output_dir": output_dir,
    },
)
res

CallToolResult(meta=None, content=[TextContent(type='text', text='/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_bsm_results.csv', annotations=None, meta=None)], structuredContent={'result': '/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_bsm_results.csv'}, isError=False)

### call_mcp_tool

### run_in_new_loop

### create_mcp_state_tool_wrapper

In [54]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [55]:
current_state = {
    "csv_file_path": csv_file_path,
    "output_dir": output_dir
}

In [56]:
from bsm_multi_agents.agents import mcp_client
importlib.reload(mcp_client)
from bsm_multi_agents.agents.mcp_client import create_mcp_state_tool_wrapper

In [57]:
bsm_state_tool = create_mcp_state_tool_wrapper(
    mcp_tool_name="calculate_bsm_to_file",
    server_script_path=server_path,
    input_arg_map={
        "csv_file_path": "input_path",
        "output_dir": "output_dir"
    },
    output_key="bsm_results_path"
)

In [58]:
result_update = bsm_state_tool(state=current_state)

In [59]:
result_update

{'bsm_results_path': '/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_bsm_results.csv'}

## Agent Level

### calculator_tool_node

#### Node Inside Level

In [60]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [61]:
from bsm_multi_agents.graph import state
importlib.reload(state)
from bsm_multi_agents.graph.state import WorkflowState

from bsm_multi_agents.agents.mcp_client import create_mcp_state_tool_wrapper

In [62]:
current_state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path
)

In [63]:
errors = current_state.get("errors", [])
[
    "csv_file_path" not in current_state or not current_state["csv_file_path"],
    "server_path" not in current_state or not current_state["server_path"]
]

[False, False]

In [64]:
server_path = current_state["server_path"]
bsm_state_fn = create_mcp_state_tool_wrapper(
    mcp_tool_name="calculate_bsm_to_file",
    server_script_path=server_path,
    input_arg_map={
        "csv_file_path": "input_path",
        "output_dir": "output_dir",
    },
    output_key="bsm_results_path",
)

greeks_state_fn = create_mcp_state_tool_wrapper(
    mcp_tool_name="calculate_greeks_to_file",
    server_script_path=server_path,
    input_arg_map={
        "csv_file_path": "input_path",
        "output_dir": "output_dir",
    },
    output_key="greeks_results_path",
)

In [65]:
bsm_update = bsm_state_fn(state=current_state)
greeks_update = greeks_state_fn(state=current_state)

In [66]:
current_state["bsm_results_path"] = bsm_update["bsm_results_path"]
current_state["greeks_results_path"] = greeks_update["greeks_results_path"]

#### Node Level

In [67]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [68]:
from bsm_multi_agents.graph import state
importlib.reload(state)
from bsm_multi_agents.graph.state import WorkflowState

from bsm_multi_agents.agents import pricing_calculator_nodes
importlib.reload(pricing_calculator_nodes)
from bsm_multi_agents.agents.pricing_calculator_nodes import pricing_calculator_tool_node

In [69]:
current_state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path
)

In [70]:
update_state = pricing_calculator_tool_node(state=current_state)
update_state

{'csv_file_path': '/Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv',
 'output_dir': '/Users/yifanli/Github/model_doc_automation/data/cache',
 'server_path': '/Users/yifanli/Github/model_doc_automation/src/bsm_multi_agents/mcp/server.py',
 'bsm_results_path': '/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_bsm_results.csv',
 'greeks_results_path': '/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_greeks_results.csv',
 'errors': []}

### calculator_agent_node

#### Node Inside Level

In [71]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")
prompt_path = os.path.join(project_path, "src/bsm_multi_agents/prompts/calculator_prompts.txt")

In [72]:
from bsm_multi_agents.graph import state
importlib.reload(state)
from bsm_multi_agents.graph.state import WorkflowState

from bsm_multi_agents.config.llm_config import get_llm
from bsm_multi_agents.prompts.loader import load_prompt

In [73]:
current_state = WorkflowState(
        csv_file_path=csv_file_path, 
        output_dir=output_dir, 
        server_path=server_path
    )

In [74]:
errors = current_state.get("errors", [])
"csv_file_path" not in current_state or not current_state["csv_file_path"]

False

In [75]:
system_prompt = """
You are a quantitative calculator agent.
You have access to tools that call an external MCP server.
Each tool reads input file paths from the shared workflow state
and writes result file paths back into the state.
Do NOT ask for raw CSV content. Always work with paths only.
You do not need to provide any arguments to the tools. Just call them with {}.
"""
user_prompt_template = load_prompt(prompt_path)
formatted_user_prompt = (
    user_prompt_template.format(csv_file_path=current_state["csv_file_path"])
    if "{csv_file_path}" in user_prompt_template
    else user_prompt_template
)
llm = get_llm()
system_msg = SystemMessage(content=system_prompt)
human_msg = HumanMessage(content=formatted_user_prompt)
ai_msg = llm.invoke([system_msg, human_msg])

In [76]:
messages = list(current_state.get("messages", []))
messages.extend([system_msg, human_msg, ai_msg])
current_state["messages"] = messages

current_state["errors"] = errors

In [77]:
current_state

{'csv_file_path': '/Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv',
 'output_dir': '/Users/yifanli/Github/model_doc_automation/data/cache',
 'server_path': '/Users/yifanli/Github/model_doc_automation/src/bsm_multi_agents/mcp/server.py',
 'messages': [SystemMessage(content='\nYou are a quantitative calculator agent.\nYou have access to tools that call an external MCP server.\nEach tool reads input file paths from the shared workflow state\nand writes result file paths back into the state.\nDo NOT ask for raw CSV content. Always work with paths only.\nYou do not need to provide any arguments to the tools. Just call them with {}.\n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Calculate Black-Scholes-Merton option prices and greeks.\n- Load the data from the /Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv.\n- Use the `calculate_bsm_to_file` tool to calculate prices.\n- Use the `calculate_greeks_to_file` tool to calc

#### Node Level

In [78]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [79]:
from bsm_multi_agents.agents import pricing_calculator_nodes
importlib.reload(pricing_calculator_nodes)
from bsm_multi_agents.agents.pricing_calculator_nodes import pricing_calculator_agent_node

from bsm_multi_agents.graph import state
importlib.reload(state)
from bsm_multi_agents.graph.state import WorkflowState

In [80]:
current_state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path
)

In [81]:
result_state = pricing_calculator_agent_node(current_state)

In [82]:
result_state

{'csv_file_path': '/Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv',
 'output_dir': '/Users/yifanli/Github/model_doc_automation/data/cache',
 'server_path': '/Users/yifanli/Github/model_doc_automation/src/bsm_multi_agents/mcp/server.py',
 'messages': [SystemMessage(content='\n    You are a quantitative calculator agent.\n    You have access to tools that call an external MCP server.\n    Each tool reads input file paths from the shared workflow state\n    and writes result file paths back into the state.\n    Do NOT ask for raw CSV content. Always work with paths only.\n    You do not need to provide any arguments to the tools. Just call them with {}.\n    ', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Calculate Black-Scholes-Merton option prices and greeks.\n- Load the data from the /Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv.\n- Use the `calculate_bsm_to_file` tool to calculate prices.\n- Use the `calculate_

### Combined Tool and Agent Levle

In [83]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [84]:
from bsm_multi_agents.graph.state import WorkflowState
from bsm_multi_agents.agents.pricing_calculator_nodes import (
    pricing_calculator_agent_node,
    pricing_calculator_tool_node,
)

In [85]:
current_state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path
)

In [86]:
current_state = pricing_calculator_agent_node(current_state)
current_state = pricing_calculator_tool_node(current_state)

In [87]:
current_state

{'csv_file_path': '/Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv',
 'output_dir': '/Users/yifanli/Github/model_doc_automation/data/cache',
 'server_path': '/Users/yifanli/Github/model_doc_automation/src/bsm_multi_agents/mcp/server.py',
 'messages': [SystemMessage(content='\n    You are a quantitative calculator agent.\n    You have access to tools that call an external MCP server.\n    Each tool reads input file paths from the shared workflow state\n    and writes result file paths back into the state.\n    Do NOT ask for raw CSV content. Always work with paths only.\n    You do not need to provide any arguments to the tools. Just call them with {}.\n    ', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Calculate Black-Scholes-Merton option prices and greeks.\n- Load the data from the /Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv.\n- Use the `calculate_bsm_to_file` tool to calculate prices.\n- Use the `calculate_

# Validator

## MCP Tool Level

### validate_greeks_to_file

In [88]:
greeks_results_path = os.path.join(project_path, "data/cache/dummy_options_greeks_results.csv")
output_dir = os.path.join(project_path, "data/cache")

In [89]:
from bsm_multi_agents.mcp import pricing_validator
importlib.reload(pricing_validator)
from bsm_multi_agents.mcp.pricing_validator import validate_greeks_to_file

In [90]:
validate_greeks_to_file(greeks_results_path, output_dir)

'/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_greeks_results_validate_results.csv'

## Call MCP Level

In [91]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
greeks_results_path = os.path.join(project_path, "data/cache/dummy_options_greeks_results.csv")
output_dir = os.path.join(project_path, "data/cache")

In [92]:
from bsm_multi_agents.agents import mcp_client
importlib.reload(mcp_client)
from bsm_multi_agents.agents.mcp_client import call_mcp_tool_async

In [93]:
res = await call_mcp_tool_async(
    tool_name="validate_greeks_to_file",
    server_script_path=server_path,
    arguments={
        "input_path": greeks_results_path,
        "output_dir": output_dir,
    },
)
res

CallToolResult(meta=None, content=[TextContent(type='text', text='/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_greeks_results_validate_results.csv', annotations=None, meta=None)], structuredContent={'result': '/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_greeks_results_validate_results.csv'}, isError=False)

## Agent Level

### pricing_validator_agent_node

#### Node Inside Level

In [94]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
greeks_results_path = os.path.join(project_path, "data/cache/dummy_options_greeks_results.csv")
output_dir = os.path.join(project_path, "data/cache")

In [95]:
from bsm_multi_agents.graph import state
importlib.reload(state)
from bsm_multi_agents.graph.state import WorkflowState

from bsm_multi_agents.agents.mcp_client import create_mcp_state_tool_wrapper

In [96]:
state = WorkflowState(
    greeks_results_path=greeks_results_path, 
    output_dir=output_dir, 
    server_path=server_path
)


In [97]:
errors = state.get("errors", [])
"greeks_results_path" not in state or not state["greeks_results_path"]

False

In [98]:
server_path = state["server_path"]
validate_greeks_state_fn = create_mcp_state_tool_wrapper(
    mcp_tool_name="validate_greeks_to_file",
    server_script_path=server_path,
    input_arg_map={
        "greeks_results_path": "input_path",
        "output_dir": "output_dir",
    },
    output_key="validate_results_path",
)
validate_update = validate_greeks_state_fn(state=state)

In [99]:
"errors" in validate_update

False

#### Node Level

In [100]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
greeks_results_path = os.path.join(project_path, "data/cache/dummy_options_greeks_results.csv")
output_dir = os.path.join(project_path, "data/cache")

In [101]:
from bsm_multi_agents.graph import state
importlib.reload(state)
from bsm_multi_agents.graph.state import WorkflowState

from bsm_multi_agents.agents import pricing_validator_agent
importlib.reload(pricing_validator_agent)
from bsm_multi_agents.agents.pricing_validator_agent import pricing_validator_agent_node

In [102]:
current_state = WorkflowState(
    greeks_results_path=greeks_results_path, 
    output_dir=output_dir, 
    server_path=server_path
)
update_state = pricing_validator_agent_node(state=current_state)
update_state

{'greeks_results_path': '/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_greeks_results.csv',
 'output_dir': '/Users/yifanli/Github/model_doc_automation/data/cache',
 'server_path': '/Users/yifanli/Github/model_doc_automation/src/bsm_multi_agents/mcp/server.py',
 'validate_results_path': '/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_greeks_results_validate_results.csv',
 'errors': []}